# Homework 2

In [192]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

## Paramethers for requests:
    1- ww_x_UNITE_ACAD - Section (Informatique?)  =  249847
    2- ww_x_PERIODE_ACAD - Semester wanted        =  978181
    3- ww_i_reportModel - General parameter =  133685247

In [193]:
unite_academique=249847
periode_academique=[978181, 978187,978195,39486325,123455150,213637754,213637922,213638028,355925344]
report_model=133685247



In [197]:
parameters={'ww_i_reportModel':'133685247','ww_i_reportModelXsl':'133685270','ww_x_UNITE_ACAD':'249847'}

In [198]:
base_all_gps='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique'

In [199]:
informatique = {'ww_x_UNITE_ACAD': '249847'}
r = requests.get(base_all_gps,params=informatique)

In [200]:
r.text

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><div></div><title></title><script type="text/javascript" src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr"></script><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style"><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><div class="filtres"><form method="GET" name="f" action="!GEDPUBLICREPORTS.filter"><input type="hidden" name="ww_b_list" value="1"><input type="hidden" name="ww_i_reportmodel" value="133685247"><input type="hidden" name="ww_c_langue" value=""><h1 id="titre">Liste des étudiants inscrits par semestre</h1><table border="0" id="format"><tr><th>Format:</th></tr><tr><td><input type="radio" name="ww_i_reportModelXsl" value="133685270" checked>html</td></t

In [201]:
semester_bachelor=['Bachelor semestre 1','Bachelor semestre 2','Bachelor semestre 3','Bachelor semestre 4','Bachelor semestre 5','Bachelor semestre 6']
semester_master=['Master semestre 1','Master semestre 2','Master semestre 3']
years=['2007-2008','2008-2009','2009-2010','2010-2011','2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017']

In [202]:
# Generate dictionary
gps_dictionary={}
for year in years:
    for semester in semester_bachelor:
        gps_dictionary['Informatique, '+year+', '+semester]=0
    for semester in semester_master:
        gps_dictionary['Informatique, '+year+', '+semester]=0

In [203]:
# Generate dictionary
main_dictionary={}
for year in years:
    main_dictionary[year]=dict()
    for item in semester_bachelor:
        main_dictionary[year][item]=0
    for item in semester_master:
        main_dictionary[year][item]=0
    

In [204]:
soup=BeautifulSoup(r.text, "lxml")

In [205]:
# Parse the parameter "periode_academique"

parse_years=soup.find_all('option')

for year in main_dictionary:
    for item in parse_years:
        if year==item.string:
            main_dictionary[year]['periode_academique']=str(item).split("\"")[1]

In [206]:
tag=soup.body

gps_list=tag.find_all('a')

In [207]:
# Process the GPS list
# Remove the ones without "Informatique", "Master" or "Bachelor"
# Hard preprocessing to fast eliminate the unwanted strings
gps_processed=[]
key1='Informatique'
key2='Bachelor'
key3='Master'
for i in gps_list:
    if key1 in i.contents[0] and (key2 in i.contents[0] or key3 in i.contents[0]):
        gps_processed.append(i)


In [209]:
# Keep only the right ones - get the GPS in the main_dictionary
for year in main_dictionary:
    for semester in main_dictionary[year]:
        for item in gps_processed:
            if 'Informatique, '+year+', '+semester==item.contents[0]:
                gps=str(item).split("'")[1].split("=")[1]
                main_dictionary[year][semester]=gps

# Get the tables

In [210]:
base_request='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html'

In [214]:
parameters={'ww_i_reportModel':'133685247','ww_i_reportModelXsl':'133685270'
            ,'ww_x_UNITE_ACAD':'249847','ww_x_PERIODE_ACAD':0,'ww_x_GPS':0}

In [225]:
for year in main_dictionary:
    parameters['ww_x_PERIODE_ACAD']=main_dictionary[year]['periode_academique']
    for semester in main_dictionary[year]:
        if semester=='periode_academique':
            continue
        parameters['ww_x_GPS']=main_dictionary[year][semester]
        req = requests.get(base_request,params=parameters)
        print(req.text)
    break

<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><link rel="stylesheet" type="text/css" href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><fieldset style="text-align:right; width:40%; position:relative; margin-right: 10px;float:right; border: 0; padding: 0 0 8px 0;"><a style="color:#990033;" href="!GEDREPORTS.html?ww_i_reportModelXsl=133685270&amp;ww_x_GPS=1744377893&amp;ww_i_reportModel=133685247&amp;ww_x_PERIODE_ACAD=213637922&amp;ww_x_UNITE_ACAD=249847">Identification pour accéder aux e-mails<br>Login to access email adresses</a></fieldset><script>
  function mailList(x) {
   var vtop = (screen.height-200)/2;
   var vleft=(screen.width-600)/2;
   var w=open("", "emaillist", "Scrollbars=1,resizable=1,width=600,height=200,top="+vtop+",left="+vleft);
   w.document.write(x);
   w.document.close();
   w.focus();
  }
 </script><h1>Extr